# Get Columns

This notebook provides an example for getting columns in Secoda. To get started, you will need to [obtain an API key](https://app.secoda.co/settings/api)

Input your API key and Secoda API endpoint URL below. The URL for the cloud instance is `https://api.secoda.co`. If you are self-hosting Secoda or on the EU instance, you will have to update the variable. For self hosted, you need to add /api/v1 to the end of the SECODA_API_URL, i.e, https://secoda.paystack.com/api/v1

In [14]:

import requests

API_KEY = ""
SECODA_API_URL = "https://api.secoda.co"

session = requests.Session()
session.headers.update(dict(
    Authorization=f"Bearer {API_KEY}"
))


def build_url(url: str):
    return f"{SECODA_API_URL}{url}"

In [ ]:
def get_paginated_entity_map(url: str):
    pages = 1
    page = 1
    entity_map = {}
    while page <= pages:
        print(f'Getting page {page}/{pages}')
        response = session.get(build_url(f"{url}?page={page}"))
        data = response.json()
        pages = data["total_pages"]
        page += 1
        for entity in data["results"]:
            entity_map[entity["id"]] = entity
    return entity_map

collection_map = get_paginated_entity_map("/collection/collections")
collection_map

In [ ]:
import pandas

fields = ['title', 'type', 'table', 'source', 'database', 'schema', 'description', 'status', 'collections', 'sql']
columns = []

page = 1
pages = 1
while page <= pages:
    print(f'Getting page {page}/{pages}')
    response = session.get(build_url(f"/table/columns?page={page}"))
    data = response.json()
    pages = data["total_pages"]
    page += 1
    for column in data["results"]:
        c = [
            column["title"],
            column["type"],
            column["table_title"],
            column["table_cluster"],
            column["table_database"],
            column["table_schema"],
            column["description"],
            "published" if column["published"] else "draft",
            ", ".join([collection_map[collection_id]["title"] for collection_id in column["collections"]]),
            column["properties"].get("custom", {}).get("sql", "")
        ]
        columns.append(c)
column_data = pandas.DataFrame(columns=fields, data=columns)
column_data.to_csv("data/columns.csv", index=False)